# TidyMongo
Make Mongo collections useful for data analysis by converting them to "tidy" pandas data frames

**References**

* [Tidy Data](http://vita.had.co.nz/papers/tidy-data.pdf)
* [Pandas](http://pandas.pydata.org/)

Querying mongo, we get lists of dictionaries.  Although this is flexible, is difficult to use for data analysis. Not to mention, our mongo collections containing nested data structures that are painful to parse.  

In [95]:
from pymongo import MongoClient

In [96]:
from pymongo import Connection
connection = Connection()
db = connection['test-database']
collection = db['test-collection']

In [ ]:
result = db['test-collection'].insert_one(
    {
        "address": {
            "street": "2 Avenue",
            "zipcode": "10075",
            "building": "1480",
            "coord": [-73.9557413, 40.7720266]
        },
        "borough": "Manhattan",
        "cuisine": "Italian",
        "grades": [
            {
                "date": datetime.strptime("2014-10-01", "%Y-%m-%d"),
                "grade": "A",
                "score": 11
            },
            {
                "date": datetime.strptime("2014-01-16", "%Y-%m-%d"),
                "grade": "B",
                "score": 17
            }
        ],
        "name": "Vella",
        "restaurant_id": "41704620"
    }
)

result = db['test-collection'].insert_one(
    {
        "address": {
            "street": "123 Main St",
            "zipcode": "555555",
            "building": "1480",
            "coord": [-73.9557413, 40.7720266]
        },
        "borough": "Durham",
        "cuisine": "American",
        "grades": [
            {
                "date": datetime.strptime("2011-07-04", "%Y-%m-%d"),
                "grade": "A",
                "score": 11
            },
            {
                "date": datetime.strptime("1983-01-25", "%Y-%m-%d"),
                "grade": "B",
                "score": 17
            }
        ],
        "name": "MadeUp Food",
        "restaurant_id": "41704621"
    }
)

In [100]:
db['test-collection'].find_one()

{u'_id': ObjectId('58514ffa77ead852f0a04819'),
 u'address': {u'building': u'1480',
  u'coord': [-73.9557413, 40.7720266],
  u'street': u'2 Avenue',
  u'zipcode': u'10075'},
 u'borough': u'Manhattan',
 u'cuisine': u'Italian',
 u'grades': [{u'date': datetime.datetime(2014, 10, 1, 0, 0),
   u'grade': u'A',
   u'score': 11},
  {u'date': datetime.datetime(2014, 1, 16, 0, 0),
   u'grade': u'B',
   u'score': 17}],
 u'name': u'Vella',
 u'restaurant_id': u'41704620'}

Yuck!!!

## Enter TidyMongo

Tool to normalize mongo collection into relational, normalized pandas dataframes

In [101]:
from tidymongo import tidy
tables = tidy(db['test-collection'].find().limit(100), 'restaurants')
tables

TidyResult(tables=[u'grades', u'address', 'restaurants'])

In [102]:
tables.restaurants.head(n=2)

,borough,cuisine,name,restaurant_id
_id,,,,
58514ffa77ead852f0a04819,Manhattan,Italian,Vella,41704620
5851505777ead852f0a0481a,Durham,American,MadeUp Food,41704621


Nested document collections are split out to new dataframes with a foreign key columns (orders)

In [103]:
tables.grades.head(n=2)

,date,grade,restaurants,score
0,2014-10-01,A,58514ffa77ead852f0a04819,11
1,2014-01-16,B,58514ffa77ead852f0a04819,17


In [104]:
tables.address.head(n=2)

,building,coord,restaurants,street,zipcode
0,1480,"[-73.9557413, 40.7720266]",58514ffa77ead852f0a04819,2 Avenue,10075
1,1480,"[-73.9557413, 40.7720266]",5851505777ead852f0a0481a,123 Main St,555555


# Example: Find mean score by restaurant

In [105]:
tables.grades.groupby('restaurants').score.mean()

restaurants
58514ffa77ead852f0a04819    14
5851505777ead852f0a0481a    14
Name: score, dtype: int64